In [101]:
import itertools
from collections import deque
from craterdetection.matching import CraterDatabase
import numpy as np
import numba as nb
from craterdetection.matching.utils import get_cliques_by_length

In [33]:
db = CraterDatabase.from_file("../data/lunar_crater_database_robbins_2018.csv",
                              diamlims=[2, 30],
                              latlims=[30, 50],
                              longlims=[30, 50],
                              radius=100
                              )
np.unique(db._crater_triads)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [5]:
np.array(db._graph.edges)

array([[ 0,  2],
       [ 0,  3],
       [ 0,  6],
       [ 0,  9],
       [ 0, 10],
       [ 0, 11],
       [ 0, 13],
       [ 1, 14],
       [ 1, 15],
       [ 2, 10],
       [ 2, 11],
       [ 2, 12],
       [ 2, 13],
       [ 3,  4],
       [ 3,  5],
       [ 3,  6],
       [ 3,  9],
       [ 3, 10],
       [ 4,  6],
       [ 5,  7],
       [ 5,  8],
       [ 5,  9],
       [ 6, 10],
       [ 7,  8],
       [ 7,  9],
       [ 8,  9],
       [ 8, 16],
       [ 9, 10],
       [ 9, 11],
       [10, 11],
       [10, 12],
       [10, 13],
       [11, 12],
       [11, 13],
       [11, 14],
       [12, 13],
       [14, 15],
       [14, 16]])

In [15]:
from itertools import combinations
import networkx as nx


def k_cliques(graph):
    # 2-cliques
    cliques = [{i, j} for i, j in graph.edges() if i != j]
    k = 2

    while cliques:
        # result
        yield k, cliques

        # merge k-cliques into (k+1)-cliques
        cliques_1 = set()
        for u, v in combinations(cliques, 2):
            print(u, v)
            w = u ^ v
            print(w)
            if len(w) == 2 and graph.has_edge(*w):
                cliques_1.add(tuple(u | w))

        # remove duplicates
        cliques = list(map(set, cliques_1))
        k += 1


def print_cliques(graph, size_k):
    for k, cliques in k_cliques(graph):
        if k == size_k:
            print('%d-cliques = %d, %s.' % (k, len(cliques), cliques))

print_cliques(db._graph, 3)

In [61]:
%%time
def get_sorted_vertices(graph, return_degree=False):
    deg = np.array(graph.degree())
    order = np.argsort(-deg[:, 1])
    if return_degree:
        return deg[order][:, 0].flatten(), deg[order][:, 1].flatten()
    else:
        return deg[order][:, 0].flatten()

vertices, degree = get_sorted_vertices(db._graph, return_degree=True)

db._graph.edges(vertices[0])

Wall time: 10 ms


array([[324,   1],
       [324,   2],
       [324,   3],
       [324, 259],
       [324, 260],
       [324, 261],
       [324, 266],
       [324, 267],
       [324, 268],
       [324, 269],
       [324, 270],
       [324, 271],
       [324, 272],
       [324, 273],
       [324, 300],
       [324, 301],
       [324, 302],
       [324, 303],
       [324, 304],
       [324, 305],
       [324, 306],
       [324, 307],
       [324, 308],
       [324, 309],
       [324, 310],
       [324, 311],
       [324, 312],
       [324, 313],
       [324, 314],
       [324, 315],
       [324, 316],
       [324, 317],
       [324, 318],
       [324, 319],
       [324, 320],
       [324, 321],
       [324, 322],
       [324, 323],
       [324, 347],
       [324, 374],
       [324, 427],
       [324, 377],
       [324, 433],
       [324, 426],
       [324, 425],
       [324, 350],
       [324, 352],
       [324, 353],
       [324, 419],
       [324, 351],
       [324, 356],
       [324, 417],
       [324,

In [91]:
%%time
number_of_triangles = int(sum(nx.triangles(db._graph).values()) / 3)
number_of_triangles

Wall time: 87 ms


85469

In [105]:
%%time
cliques = np.empty((number_of_triangles, 3), int)
i = 0
for c in nx.enumerate_all_cliques(db._graph):
    if len(c) == 3:
        cliques[i] = np.array(c)
        i += 1

    if len(c) > 3:
        break
cliques

Wall time: 5.13 s


array([[  0,   1, 348],
       [  0,   1, 349],
       [  0,   1, 350],
       ...,
       [431, 432, 435],
       [431, 434, 435],
       [432, 434, 435]])

In [103]:
%%time
get_cliques_by_length(db._graph, 3)

Wall time: 4.59 s


[[0, 1, 348],
 [0, 1, 349],
 [0, 1, 350],
 [0, 1, 351],
 [0, 1, 352],
 [0, 1, 353],
 [0, 1, 354],
 [0, 1, 355],
 [0, 1, 356],
 [0, 1, 374],
 [0, 1, 375],
 [0, 1, 376],
 [0, 1, 377],
 [0, 1, 378],
 [0, 1, 379],
 [0, 1, 380],
 [0, 1, 381],
 [0, 1, 425],
 [0, 1, 433],
 [0, 348, 349],
 [0, 348, 350],
 [0, 348, 351],
 [0, 348, 352],
 [0, 348, 353],
 [0, 348, 354],
 [0, 348, 355],
 [0, 348, 356],
 [0, 348, 374],
 [0, 348, 375],
 [0, 348, 376],
 [0, 348, 377],
 [0, 348, 378],
 [0, 348, 379],
 [0, 348, 381],
 [0, 348, 425],
 [0, 348, 433],
 [0, 349, 350],
 [0, 349, 351],
 [0, 349, 352],
 [0, 349, 353],
 [0, 349, 354],
 [0, 349, 355],
 [0, 349, 356],
 [0, 349, 374],
 [0, 349, 375],
 [0, 349, 376],
 [0, 349, 377],
 [0, 349, 378],
 [0, 349, 379],
 [0, 349, 380],
 [0, 349, 381],
 [0, 349, 425],
 [0, 349, 433],
 [0, 350, 351],
 [0, 350, 352],
 [0, 350, 353],
 [0, 350, 354],
 [0, 350, 355],
 [0, 350, 356],
 [0, 350, 374],
 [0, 350, 375],
 [0, 350, 376],
 [0, 350, 377],
 [0, 350, 378],
 [0, 350, 379]

In [68]:
def enumerate_cliques(G):
    index = {}
    nbrs = {}
    for u in G:
        index[u] = len(index)
        # Neighbors of u that appear after u in the iteration order of G.
        nbrs[u] = {v for v in G[u] if v not in index}

    queue = deque(([u], sorted(nbrs[u], key=index.__getitem__)) for u in G)
    # Loop invariants:
    # 1. len(base) is nondecreasing.
    # 2. (base + cnbrs) is sorted with respect to the iteration order of G.
    # 3. cnbrs is a set of common neighbors of nodes in base.
    while queue:
        base, cnbrs = map(list, queue.popleft())
        yield base
        for i, u in enumerate(cnbrs):
            # Use generators to reduce memory consumption.
            queue.append(
                (
                    itertools.chain(base, [u]),
                    filter(nbrs[u].__contains__, itertools.islice(cnbrs, i + 1, None)),
                )
            )

90131

In [114]:
G = db._graph

index = {}
nbrs = {}
for u in G:
    index[u] = len(index)
    # Neighbors of u that appear after u in the iteration order of G.
    nbrs[u] = {v for v in G[u] if v not in index}

queue = deque(([u], sorted(nbrs[u], key=index.__getitem__)) for u in G)
# Loop invariants:
# 1. len(base) is nondecreasing.
# 2. (base + cnbrs) is sorted with respect to the iteration order of G.
# 3. cnbrs is a set of common neighbors of nodes in base.
base, cnbrs = map(list, queue.popleft())
print(base)
for i, u in enumerate(cnbrs):
    # Use generators to reduce memory consumption.
    queue.append(
        (
            itertools.chain(base, [u]),
            filter(nbrs[u].__contains__, itertools.islice(cnbrs, i + 1, None)),
        )
    )

[0]


In [121]:
queue.count(6)

0